### **Splatoon 3 side order  in depth analysis**

Side order is the latest released single player dlc for splatoon 3 on the nintendo switch. The gamemode is structured in a roadlike game, with progress being tracked by the amount of floors the player has went up.
there are a total of 30 floors after the first climb of 10 floors. Each floor will progress in difficulty, but does allow the oppurtunity for the player to enhance their weapons & abilities.

<img src="../images/side-order-main-image.webp" alt="Side order image" width="300">

These upgrades can be acquired by collecting colorchips. There are 6 categories in wich a colorchip can increase different damage, movement, mobility or even the pearl drone enhancements. The last importent feature is the Marina hacks that you can boost by (prlz) wich are the ingame currency for leveling them. Marina's hacks mostly involve player durability, lives and armor statistics.

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from plotly.subplots import make_subplots

The first thing we will look at are the damage types and their increasing scalability. They range from any source of damage that can be dealt by either the player or pearl drone. Such as main weapon, sub weapon, special weapon, pearl drone abilities and stage mechanics like the towers sprinkler.

In [44]:
powerchips = pd.read_csv('../data/colorchips & damage/colorchips/power_colorchips.csv')
rangechips = pd.read_csv('../data/colorchips & damage/colorchips/range_colorchips_main_ink.csv')
mobilitychips = pd.read_csv('../data/colorchips & damage/colorchips/mobility_colorchips.csv')
supportchips = pd.read_csv('../data/colorchips & damage/colorchips/support_colorchips.csv')
luckychips = pd.read_csv('../data/colorchips & damage/colorchips/lucky_colorchips.csv')
dronechips = pd.read_csv('../data/colorchips & damage/colorchips/drone_colorchips.csv')

# Exclude certain columns from mobilitychips
exclude_columns_mchips = ['Extra Dodge Roll', 'Charge Storage', 'Squid Attack', 'Dodge Roll Attack', 'Lunge Attack', 'Mobile Special Charge', 'Mobile Drone Gauge', 'Extra comments']
mobilitychip_accepted_columns = [col for col in mobilitychips.columns[2:-1] if not any(exclude_col in col for exclude_col in exclude_columns_mchips)]

# Convert data types to scale in percentage or in point costs
def process_colorchips(df, exclude_columns, columns):
    for col in columns:
        if col not in exclude_columns:
            if df[col].dtype == 'object' and '%' in df[col].iloc[0]:
                df[col] = df[col].str.rstrip('%').astype('float')
            else:
                df[col] = df[col].astype('float')
    return df

# Process each colorchip dataset
powerchips = process_colorchips(powerchips, ['id'], powerchips.columns[2:-1])
rangechips = process_colorchips(rangechips, ['id'], rangechips.columns[4:-2])
mobilitychips = process_colorchips(mobilitychips, ['id'], mobilitychip_accepted_columns)
supportchips = process_colorchips(supportchips, ['id'], supportchips.columns[2:-5])
luckychips = process_colorchips(luckychips, ['id'], luckychips.columns[2:-3])
dronechips = process_colorchips(dronechips, ['id'], dronechips.columns[2:])

# Create the initial plot
fig = make_subplots(rows=1, cols=1)
trace_count = 0

# Define categories and corresponding image sources
categories = [
    (powerchips, powerchips.columns[2:-1], 'Power Colorchip', '../images/colorchips/power_colorchip.png'),
    (rangechips, rangechips.columns[4:-2], 'Range Colorchip', '../images/colorchips/range_colorchip.png'),
    (mobilitychips, mobilitychip_accepted_columns, 'Mobility Colorchip', '../images/colorchips/mobility_colorchip.png'),
    (supportchips, supportchips.columns[2:-5], 'Support Colorchip', '../images/colorchips/support_colorchip.png'),
    (luckychips, luckychips.columns[2:-3], 'Lucky Colorchip', '../images/colorchips/lucky_colorchip.png'),
    (dronechips, dronechips.columns[2:], 'Drone Colorchip', '../images/colorchips/drone_colorchip.png'),
]

# Add traces to the plot
trace_visibility = []
for df, columns, name, _ in categories:
    start_idx = trace_count
    for col in columns:
        fig.add_trace(go.Scatter(
            x=df['id'],
            y=df[col],
            mode='lines+markers',
            name=col,
            hoverinfo='text',
            visible=(name == 'Power Colorchip'),
            text=[f'Stage: {x}<br>{col}: {y}%' for x, y in zip(df['id'], df[col])]
        ))
        trace_count += 1
    trace_visibility.append((start_idx, trace_count - 1))

# Function to create update arguments
def create_update_args(start, end, y_title, y_ticktext, plot_title, image_source):
    visibility = [False] * trace_count
    for i in range(start, end + 1):
        visibility[i] = True
    return [
        {'visible': visibility},
        {
            'yaxis.title.text': y_title,
            'yaxis.ticktext': y_ticktext,
            'title.text': plot_title,
            'images': [{'source': image_source, 'xref': "paper", 'yref': "paper", 'x': 0.8, 'y': 0.7, 'sizex': 0.4, 'sizey': 0.4, 'xanchor': "center", 'yanchor': "middle", 'opacity': 0.5}]
        }
    ]

# Add a dropdown menu to switch between categories
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(label='Power Colorchip', method='update',
                     args=create_update_args(0, len(powerchips.columns[2:-1]) - 1,
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Power Colorchip</b>',
                                             '../images/colorchips/power_colorchip.png')),
                dict(label='Range Colorchip', method='update',
                     args=create_update_args(trace_visibility[1][0], trace_visibility[1][1],
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Range Colorchip</b>',
                                             '../images/colorchips/range_colorchip.png')),
                dict(label='Mobility Colorchip', method='update',
                     args=create_update_args(trace_visibility[2][0], trace_visibility[2][1],
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Mobility Colorchip</b>',
                                             '../images/colorchips/mobility_colorchip.png')),
                dict(label='Support Colorchip', method='update',
                     args=create_update_args(trace_visibility[3][0], trace_visibility[3][1],
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Support Colorchip</b>',
                                             '../images/colorchips/support_colorchip.png')),
                dict(label='Lucky Colorchip', method='update',
                     args=create_update_args(trace_visibility[4][0], trace_visibility[4][1],
                                             '<b>Increment in (%)</b>',
                                             [f"{tick}%" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Lucky Colorchip</b>',
                                             '../images/colorchips/lucky_colorchip.png')),
                dict(label='Drone Colorchip', method='update',
                     args=create_update_args(trace_visibility[5][0], trace_visibility[5][1],
                                             '<b>Increment in (p)</b>',
                                             [f"{tick}p" for tick in range(100, 1001, 100)],
                                             '<b>Upgrades per Drone Colorchip</b>',
                                             '../images/colorchips/drone_colorchip.png'))
            ],
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=1.285,
            xanchor='right',
            y=0.875,
            yanchor='bottom'
        )
    ]
)

# Initial layout
fig.update_layout(
    title={
        'text': '<b>Upgrades per Power Colorchip</b>',
        'y': 0.965,
        'x': 0.4,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'size': 14,
            'family': 'Arial',
            'color': 'black',
        }
    },
    xaxis=dict(
        title='<b>Stage Upgrade Number</b>',
        tickmode='array',
        tickvals=list(range(1, 11)),
        ticktext=list(range(1, 11)),
        titlefont=dict(family='Arial', size=12, color='black'),
        range=[1, 10.1]
    ),
    yaxis=dict(
        title=dict(
            text='<b>Increment in (%)</b>',
            standoff=0.2  # Adjust this value to move the title to the right
        ),
        tickmode='array',
        tickvals=list(range(100, 1001, 100)),
        ticktext=[f"{tick}%" for tick in range(100, 1001, 100)],
        titlefont=dict(family='Arial', size=12, color='black')
    ),
    legend=dict(
        x=1.01,
        y=0.8,
        traceorder='normal',
        font=dict(family='Arial', size=12, color='black'),
        bgcolor='rgba(0,0,0,0)',
        bordercolor='rgba(0,0,0,0)'
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    hovermode='closest',
    width=900 
)

# Add initial image
fig.add_layout_image(
    dict(
        source='../images/colorchips/power_colorchip.png',
        xref="paper", yref="paper",
        x=0.8, y=0.7,
        sizex=0.4, sizey=0.4,
        xanchor="center", yanchor="middle",
        opacity=0.5
    )
)

# Show the plot
fig.show()